# Standardization

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../data/DAGs.csv')
df.head()

,Author,Exposure,Outcome,Direction,Strength,ID,Status
0,NaN,Diabetes,Ischemic stroke,Increase,6.0,1,Final
1,NaN,age,Ischemic stroke,Increase,6.0,1,Final
2,NaN,age,Diabetes,Increase,5.0,1,Final
3,NaN,Sex (Male),Ischemic stroke,Increase,6.0,1,Final
4,NaN,Hypertension,Ischemic stroke,Increase,6.0,1,Final


### Getting list of unique terms

In [3]:
terms = pd.concat([df.Exposure, df.Outcome], ignore_index=True).unique()
pd.DataFrame(terms, columns=['terms']).to_csv('terms.csv', index=False)

### Import Usagi mapping and Excel notes

* `source_code_description` - Original term name
* `target_concept_id` - OHDSI ATHENA ID for term 
* `time` - If a concept was used multiple times to keep track of its position in the DAG
* `direction` - Whether the direction in the DAG needs to be switched (e.g. female is coded as male so the direction needs to be changed from increase to decrease or vice versa in the original DAG) 

In [4]:
mapping = pd.read_csv('usagi_export.csv', dtype={'target_concept_id': str})[['source_code_description','target_concept_id']]
mapping.replace('0',np.nan, inplace=True)

# Add non standard snomed terms
non_standard_additions = pd.read_excel('terms_mapped_to_codes.xlsx', sheet_name='non_standard_concepts', dtype={'target_concept_id': str})
mapping = mapping.merge(non_standard_additions, on='source_code_description', how='left', suffixes=['','_non_standard'])
mapping['target_concept_id'] = mapping['target_concept_id'].fillna(mapping['target_concept_id_non_standard'])
mapping = mapping.drop('target_concept_id_non_standard', axis=1)

# Add annotations of time and direction
annotations = pd.read_excel('terms_mapped_to_codes.xlsx', sheet_name='terms')[['term','time','direction']]
mapping = mapping.merge(annotations, left_on='source_code_description', right_on='term', how='left').drop('term', axis=1)

mapping.head()

,source_code_description,target_concept_id,time,direction
0,race,4013886,NaN,NaN
1,Lacunes,4046360,NaN,NaN
2,chemotherapy,4273629,NaN,NaN
3,malformations,4079975,NaN,NaN
4,kidney disease,198124,NaN,NaN


### Missing Mappings

In [5]:
mapping[mapping.target_concept_id.isna()]

,source_code_description,target_concept_id,time,direction
14,peripheral immune competence,NaN,NaN,NaN
16,microglial proliferation,NaN,NaN,NaN
44,nonatherosclerotic abnormalities,NaN,NaN,NaN
52,LMV,NaN,NaN,NaN
53,B-cell infiltration,NaN,NaN,NaN
75,Cardiac Remodelling,NaN,NaN,NaN
92,Other Cardiac Pathologies,NaN,NaN,NaN
104,production of reactive oxygen species,NaN,NaN,NaN
105,loss of brain structural integrity,NaN,NaN,NaN
113,collateralization,NaN,NaN,NaN


For now, fill missing mappings with original term.

In [6]:
mapping['target_concept_id'] = mapping.target_concept_id.fillna('source_code_description')

### Match mapping codes with Athena database

In [7]:
concept_columns = ['concept_id',
                   'concept_name',
                   'domain_id',
                   'vocabulary_id',
                   'concept_class_id',
                   'concept_code'
                   ]
athena_concepts = pd.read_csv('athena_vocabulary/CONCEPT.csv', sep='\t', dtype={'concept_id': str, 'concept_code': str}, low_memory=False)[concept_columns]

mapped_terms = mapping.merge(athena_concepts, how='left', left_on='target_concept_id', right_on='concept_id').drop('target_concept_id', axis=1)
mapped_terms.head()

,source_code_description,time,direction,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,concept_code
0,race,NaN,NaN,4013886,Race,Observation,SNOMED,Observable Entity,103579009
1,Lacunes,NaN,NaN,4046360,Lacunar infarction,Condition,SNOMED,Disorder,230698000
2,chemotherapy,NaN,NaN,4273629,Chemotherapy,Procedure,SNOMED,Procedure,367336001
3,malformations,NaN,NaN,4079975,Congenital malformation,Condition,SNOMED,Disorder,276654001
4,kidney disease,NaN,NaN,198124,Kidney disease,Condition,SNOMED,Disorder,90708001


### Apply direction switches to arrows
Pairs that need switching:
* Blood-brain-barrier breakdown (vs. integrity)
* Cognitive function / cognition (vs. impairment)
* Female (vs. male)
* Immobility (vs. mobility)
* Low socioeconomic status (vs. socioeconomic status)
* Physical inactivity (vs. exercise)
* reduced quality of life (vs. quality of life)

In [8]:
switch_terms = mapped_terms[mapped_terms.direction=='switch'].source_code_description

In [9]:
def flip_direction(direction):
    return "Decrease" if direction == "Increase" else "Increase"

def apply_switch_logic(row):
    # Count how many terms in `switch_terms` appear in `Exposure` or `Outcome`
    match_count = sum(term in row["Exposure"] or term in row["Outcome"] for term in switch_terms)
    
    # If there is exactly one match, flip the direction
    if match_count == 1:
        return flip_direction(row["Direction"])
    # If there are two matches (both terms are present), keep the direction unchanged
    else:
        return row["Direction"]
    
df["Direction"] = df.apply(apply_switch_logic, axis=1)

### Will create separate nodes for compound terms with same arrows

In [10]:
mapped_terms[mapped_terms.source_code_description.duplicated(keep=False)]

,source_code_description,time,direction,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,concept_code
11,vessel stenosis/occlusion,NaN,NaN,4217691,Stenosis,Observation,SNOMED,Morph Abnormality,415582006
12,vessel stenosis/occlusion,NaN,NaN,4178903,Complete obstruction,Observation,SNOMED,Morph Abnormality,50173008
121,Carotid Endarterectomy / Angioplasty / stenting,NaN,NaN,4283095,Carotid endarterectomy,Procedure,SNOMED,Procedure,66951008
122,Carotid Endarterectomy / Angioplasty / stenting,NaN,NaN,4178631,Angioplasty of carotid artery,Procedure,SNOMED,Procedure,429287007
123,Carotid Endarterectomy / Angioplasty / stenting,NaN,NaN,4050288,Insertion of carotid artery stent,Procedure,SNOMED,Procedure,233405004
138,atherosclerosis/thrombosis,NaN,NaN,4306703,Atherosclerosis,Observation,SNOMED,Morph Abnormality,38716007
139,atherosclerosis/thrombosis,NaN,NaN,4231363,Thrombosis,Condition,SNOMED,Disorder,439127006
268,preeclampsia and other pregancy related compli...,NaN,NaN,439393,Pre-eclampsia,Condition,SNOMED,Disorder,398254007
269,preeclampsia and other pregancy related compli...,NaN,NaN,42538946,Hypertension complicating pregnancy,Condition,SNOMED,Disorder,82771000119102
270,preeclampsia and other pregancy related compli...,NaN,NaN,37018765,Gestational diabetes mellitus complicating pre...,Condition,SNOMED,Disorder,40801000119106


Option 1: Dropping duplicates

In [21]:
mapped_terms_no_dup = mapped_terms[~mapped_terms.source_code_description.duplicated()]

Option 2: Creating new nodes with same connections for duplicates

### Combining mapping with original DAG data

In [22]:
mapped_terms_dict = mapped_terms_no_dup.set_index('source_code_description')['concept_name'].to_dict()

In [23]:
df['Exposure'] = df.Exposure.replace(mapped_terms_dict)
df['Outcome'] = df.Outcome.replace(mapped_terms_dict)
df.head()

,Author,Exposure,Outcome,Direction,Strength,ID,Status
0,NaN,Diabetes mellitus,Ischemic stroke,Increase,6.0,1,Final
1,NaN,Age factor,Ischemic stroke,Increase,6.0,1,Final
2,NaN,Age factor,Diabetes mellitus,Increase,5.0,1,Final
3,NaN,Male,Ischemic stroke,Increase,6.0,1,Final
4,NaN,Hypertensive disorder,Ischemic stroke,Increase,6.0,1,Final


In [25]:
df.to_csv('../data/DAGs_standardized.csv', index=False)